In [1]:
!pip install unsloth

In [2]:
!python -m pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8bu14ynk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8bu14ynk
  Resolved https://github.com/huggingface/transformers to commit 9be4728af8bec48073ae841881d7f4e2ac3521d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10963677 sha256=027197aeeafdf274c145a00582713c2154e9c21aca59977658d683bef5a9a483
  Stored in directory: /tmp/pip-ephem-wheel-cache-scdi7fw6/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [3]:
!pip install torch

In [4]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [5]:
  from unsloth import FastVisionModel # FastLanguageModel for LLMs
  import torch

  # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
  fourbit_models = [
      "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
      "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
      "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
      "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

      "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
      "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

      "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
      "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
      "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

      "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
      "unsloth/llava-1.5-7b-hf-bnb-4bit",
  ] # More models at https://huggingface.co/unsloth

  model, tokenizer = FastVisionModel.from_pretrained(
      "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit",
      load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
      use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
  )

<ipython-input-5-9850a2d8ff41>:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastVisionModel # FastLanguageModel for LLMs


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.17: Fast Qwen2 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [6]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [7]:
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit",
#     torch_dtype="auto",
#     device_map="auto",
# )

# Use the same checkpoint for the processor as for the model
# processor = AutoProcessor.from_pretrained(
#     "unsloth/Qwen2.5-VL-7B-Instruct-unsloth-bnb-4bit"  # Changed to match the model checkpoint
# )
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [8]:
from datasets import load_dataset
dataset = load_dataset("Rocktim/EXAMS-V", split="train")
print(f"Dataset loaded: {dataset}")

# Save the dataset to disk
dataset.save_to_disk("dataset")

README.md:   0%|          | 0.00/6.82k [00:00<?, ?B/s]

train-00000-of-00009.parquet:   0%|          | 0.00/450M [00:00<?, ?B/s]

train-00001-of-00009.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

train-00002-of-00009.parquet:   0%|          | 0.00/664M [00:00<?, ?B/s]

train-00003-of-00009.parquet:   0%|          | 0.00/571M [00:00<?, ?B/s]

...

test-00002-of-00003.parquet:   0%|          | 0.00/565M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16494 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4797 [00:00<?, ? examples/s]

Dataset loaded: Dataset({
    features: ['image', 'sample_id', 'answer_key', 'type', 'grade', 'subject', 'subject_grouped', 'language', 'chemical_structure', 'table', 'figure', 'graph'],
    num_rows: 16494
})


Saving the dataset (0/16 shards):   0%|          | 0/16494 [00:00<?, ? examples/s]

In [9]:
# prompt: clear gpu and ram not in use

import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [10]:
import os
import json
from PIL import Image
from datasets import load_from_disk

# Load dataset
dataset_dir = "/content/dataset"  # Change this if needed
dataset = load_from_disk(dataset_dir)

# Define output folder and create it if necessary
output_folder = "/content/downloaded_images"
os.makedirs(output_folder, exist_ok=True)

# Maximum allowed pixels
MAX_PIXELS = 1280 * 28 * 28

# Initialize a dictionary to store image paths and their corresponding answer keys
results_dict = {}

# Process up to 200 images from the dataset
num_samples = min(200, len(dataset))
print(f"Processing {num_samples} images...")

for i, example in enumerate(dataset.select(list(range(num_samples)))):
    try:
        # Retrieve and convert the image to RGB
        img = example["image"]
        if not isinstance(img, Image.Image):
            print(f"Warning: Item {i} does not contain a valid image. Skipping.")
            continue
        img = img.convert("RGB")

        # Calculate new dimensions while maintaining aspect ratio
        width, height = img.size
        scale_factor = min((MAX_PIXELS / (width * height)) ** 0.5, 1.0)
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)

        # Resize the image using LANCZOS resampling
        img_resized = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        # Define the new filename and save the resized image
        image_filename = f"image_{i}.png"
        image_path = os.path.join(output_folder, image_filename)
        img_resized.save(image_path, format="PNG", quality=100)

        # Save the mapping of image path to answer key
        answer_key = example["answer_key"]
        results_dict[image_path] = answer_key

        print(f"✅ Resized & saved: {image_filename} ({new_width}x{new_height}) | Answer: {answer_key}")

    except Exception as e:
        print(f"❌ Error processing image {i}: {e}")

# Save the results dictionary to a JSON file
results_json_path = os.path.join(output_folder, "results.json")
with open(results_json_path, "w") as json_file:
    json.dump(results_dict, json_file, indent=4)

print(f"\nResults JSON saved at: {results_json_path}")


Processing 200 images...
✅ Resized & saved: image_0.png (1158x866) | Answer: A
✅ Resized & saved: image_1.png (1410x711) | Answer: C
✅ Resized & saved: image_2.png (1364x735) | Answer: D
✅ Resized & saved: image_3.png (1253x800) | Answer: A
✅ Resized & saved: image_4.png (1226x818) | Answer: B
✅ Resized & saved: image_5.png (1206x831) | Answer: D
✅ Resized & saved: image_6.png (1219x823) | Answer: C
✅ Resized & saved: image_7.png (1308x766) | Answer: A
✅ Resized & saved: image_8.png (1538x652) | Answer: B
✅ Resized & saved: image_9.png (1446x693) | Answer: B
✅ Resized & saved: image_10.png (1690x593) | Answer: B
✅ Resized & saved: image_11.png (1399x716) | Answer: A
✅ Resized & saved: image_12.png (1814x553) | Answer: C
✅ Resized & saved: image_13.png (1248x803) | Answer: B
✅ Resized & saved: image_14.png (1561x642) | Answer: D
✅ Resized & saved: image_15.png (1558x644) | Answer: B
✅ Resized & saved: image_16.png (1553x645) | Answer: A
✅ Resized & saved: image_17.png (1492x672) | Answe

In [11]:
# prompt: clear gpu and ram not in use

import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [12]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [15]:
import os
import gc
import json
import re
import torch
from PIL import Image as PILImage

#####################################
# 1) Define your folder of images
#####################################
image_folder = "/content/downloaded_images"  # Update as needed
output_filename = "output_raw.jsonl"                # JSONL file to save results


#####################################
# 2) Updated parser function
#####################################
def parse_model_output(output_text):
    # Remove enclosing triple backticks if present.
    stripped_text = output_text.strip()
    if stripped_text.startswith("```"):
        lines = stripped_text.splitlines()
        if lines[0].strip().startswith("```"):
            lines = lines[1:]
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        stripped_text = "\n".join(lines).strip()

    # Use regex to extract the JSON block between the first '{' and the last '}'
    json_match = re.search(r'({.*})', stripped_text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        json_str = stripped_text

    try:
        data = json.loads(json_str)
    except Exception as e:
        # If parsing fails, attempt to find a one-letter answer in double quotes
        answer_match = re.search(r'"([A-Da-d])"', output_text)
        if answer_match:
            extracted_answer = answer_match.group(1).upper()
        else:
            extracted_answer = "N/A"
        data = {
            "question_native": "N/A",
            "question_en": "N/A",
            "choices_native": [],
            "choices_en": [],
            "correct_answer": extracted_answer,
            "step_1": "N/A",
            "step_2": "N/A",
            "step_3": "N/A",
            "step_4": "N/A",
            "final_explanation": output_text,
        }
    return data

#####################################
# 3) Function to generate reasoning
#####################################
def generate_reasoning(image_path, correct_answer):
    # Load the image
    image = PILImage.open(image_path)

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {
                    "type": "text",
                    "text": (
                        "I am providing an image that contains a question with multiple choice options. "
                        "Please extract all details from the image and output your answer in a strict JSON format. "
                        "Your JSON should include the following keys:\n\n"
                        "• question_native: The question in the native language (if available).\n"
                        "• question_en: The question in English.\n"
                        "• choices_native: A list of answer options in the native language.\n"
                        "• choices_en: A list of answer options in English.\n"
                        "• correct_answer: The correct answer as indicated in the image. Please ensure this value is one of 'A', 'B', 'C', or 'D'.\n"
                        "• step_1: Your first step of reasoning (e.g., understanding the question).\n"
                        "• step_2: Your second step of reasoning (e.g., analyzing the provided information).\n"
                        "• step_3: Provide your reasoning as a single plain string without any bullet points, lists, or markdown formatting. "
                        "  If you need to list items, separate them with commas, ensuring the entire value remains valid JSON.\n"
                        "• step_4: Your fourth step of reasoning (e.g., reaching the conclusion).\n"
                        "• final_explanation: A comprehensive explanation summarizing your reasoning.\n\n"
                        "Example output:\n"
                        "{\"question_native\": \"ما اسم هذا المركب؟\", "
                        "\"question_en\": \"What is the name of this compound?\", "
                        "\"choices_native\": [\"إيثان\", \"إيثين\", \"إيثانال\", \"إيثانول\"], "
                        "\"choices_en\": [\"Ethane\", \"Ethene\", \"Ethanal\", \"Ethanol\"], "
                        "\"correct_answer\": \"A\", "
                        "\"step_1\": \"Step 1: Understanding the question...\", "
                        "\"step_2\": \"Step 2: Analyzing the information...\", "
                        "\"step_3\": \"Step 3: Evaluating each option ensuring valid JSON...\", "
                        "\"step_4\": \"Step 4: Reaching the conclusion...\", "
                        "\"final_explanation\": \"Final explanation: Based on the analysis, the correct answer is Ethane.\"}\n\n"
                        "Make sure your output is valid JSON."
                    )
                }
            ]
        }
    ]

    # 'processor' and 'model' are assumed to be defined elsewhere in your code
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt"
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    del image
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    output_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]

    # Print only the AI Reasoning Output
    print("\n🔍 **AI Reasoning Output:**\n")
    print(output_text)

    del inputs, output_ids, generated_ids
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return output_text

#####################################
# 4) Process each image & write JSONL output
#####################################
with open(output_filename, "w", encoding="utf-8") as f_out:
    for filename in sorted(os.listdir(image_folder)):
        if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        image_path = os.path.join(image_folder, filename)

        # Get the base filename (e.g. "resized_image_0.png")
        base_name = os.path.basename(filename)
        # Remove the "resized_" prefix if present so it matches the original image name
        if base_name.startswith("resized_"):
            base_name = base_name.replace("resized_", "", 1)

        # Reconstruct the original path as stored in the JSON mapping (if needed)
        original_path_in_json = os.path.join("/content/downloaded_images", base_name)
        # # Optionally, you could still get an answer from answers_dict:
        # _ = answers_dict.get(original_path_in_json, "N/A")

        # Generate reasoning (only AI Reasoning Output is printed from within the function)
        ai_output_text = generate_reasoning(image_path, _)
        parsed_data = parse_model_output(ai_output_text)

        # Use the correct answer from the model's output.
        model_correct_answer = parsed_data.get("correct_answer", "N/A")
        if model_correct_answer not in ["A", "B", "C", "D"]:
            final_answer = "N/A"
        else:
            final_answer = model_correct_answer

        # Create a new dictionary with only the correct_answer and image_path
        result = {
            "image_path": image_path,
            "correct_answer": final_answer,
        }

        # Print the result
        print("\nResult:")
        print(json.dumps(result, ensure_ascii=False))

        # Save the result in the output JSONL file.
        f_out.write(json.dumps(result, ensure_ascii=False) + "\n")

        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

print(f"\n✅ Done processing all images. Results saved in '{output_filename}'.")



🔍 **AI Reasoning Output:**

```json
{
  "question_native": "ما اسم هذا المركب؟",
  "question_en": "What is the name of this compound?",
  "choices_native": [
    "إيثان",
    "إيثين",
    "إيثانال",
    "إيثانول"
  ],
  "choices_en": [
    "Ethane",
    "Ethene",
    "Ethanal",
    "Ethanol"
  ],
  "correct_answer": "A",
  "step_1": "The question asks for the name of the organic compound shown in the diagram.",
  "step_2": "The diagram shows two carbon atoms connected by a single bond, each bonded to three hydrogen atoms.",
  "step_3": "Option A: Ethane - Two carbon atoms bonded together with no additional functional groups.\nOption B: Ethene - Two carbon atoms bonded together with a double bond.\nOption C: Ethanal - Two carbon atoms bonded together with an aldehyde group (-CHO).\nOption D: Ethanol - Two carbon atoms bonded together with an alcohol group (-OH).",
  "step_4": "Based on the structure, it matches the structure of ethane, which has two carbon atoms bonded together with no

KeyboardInterrupt: 

In [16]:
import os
import gc
import json
import re
import torch
from PIL import Image as PILImage

#####################################
# 1) Define your folder of images
#####################################
image_folder = "/content/downloaded_images"  # Update as needed
output_filename = "output_raw.jsonl"           # JSONL file to save results

#####################################
# 2) Load already processed results (if any)
#####################################
processed_images = set()
if os.path.exists(output_filename):
    with open(output_filename, "r", encoding="utf-8") as f_in:
        for line in f_in:
            try:
                result = json.loads(line)
                processed_images.add(result.get("image_path"))
            except json.JSONDecodeError:
                continue

#####################################
# 3) Updated parser function
#####################################
def parse_model_output(output_text):
    # Remove enclosing triple backticks if present.
    stripped_text = output_text.strip()
    if stripped_text.startswith("```"):
        lines = stripped_text.splitlines()
        if lines[0].strip().startswith("```"):
            lines = lines[1:]
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        stripped_text = "\n".join(lines).strip()

    # Use regex to extract the JSON block between the first '{' and the last '}'
    json_match = re.search(r'({.*})', stripped_text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        json_str = stripped_text

    try:
        data = json.loads(json_str)
    except Exception as e:
        # If parsing fails, attempt to find a one-letter answer in double quotes
        answer_match = re.search(r'"([A-Da-d])"', output_text)
        if answer_match:
            extracted_answer = answer_match.group(1).upper()
        else:
            extracted_answer = "N/A"
        data = {
            "question_native": "N/A",
            "question_en": "N/A",
            "choices_native": [],
            "choices_en": [],
            "correct_answer": extracted_answer,
            "step_1": "N/A",
            "step_2": "N/A",
            "step_3": "N/A",
            "step_4": "N/A",
            "final_explanation": output_text,
        }
    return data

#####################################
# 4) Function to generate reasoning
#####################################
def generate_reasoning(image_path, correct_answer):
    # Load the image
    image = PILImage.open(image_path)

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {
                    "type": "text",
                    "text": (
                        "I am providing an image that contains a question with multiple choice options. "
                        "Please extract all details from the image and output your answer in a strict JSON format. "
                        "Your JSON should include the following keys:\n\n"
                        "• question_native: The question in the native language (if available).\n"
                        "• question_en: The question in English.\n"
                        "• choices_native: A list of answer options in the native language.\n"
                        "• choices_en: A list of answer options in English.\n"
                        "• correct_answer: The correct answer as indicated in the image. Please ensure this value is one of 'A', 'B', 'C', or 'D'.\n"
                        "• step_1: Your first step of reasoning (e.g., understanding the question).\n"
                        "• step_2: Your second step of reasoning (e.g., analyzing the provided information).\n"
                        "• step_3: Provide your reasoning as a single plain string without any bullet points, lists, or markdown formatting. "
                        "  If you need to list items, separate them with commas, ensuring the entire value remains valid JSON.\n"
                        "• step_4: Your fourth step of reasoning (e.g., reaching the conclusion).\n"
                        "• final_explanation: A comprehensive explanation summarizing your reasoning.\n\n"
                        "Example output:\n"
                        "{\"question_native\": \"ما اسم هذا المركب؟\", "
                        "\"question_en\": \"What is the name of this compound?\", "
                        "\"choices_native\": [\"إيثان\", \"إيثين\", \"إيثانال\", \"إيثانول\"], "
                        "\"choices_en\": [\"Ethane\", \"Ethene\", \"Ethanal\", \"Ethanol\"], "
                        "\"correct_answer\": \"A\", "
                        "\"step_1\": \"Step 1: Understanding the question...\", "
                        "\"step_2\": \"Step 2: Analyzing the information...\", "
                        "\"step_3\": \"Step 3: Evaluating each option ensuring valid JSON...\", "
                        "\"step_4\": \"Step 4: Reaching the conclusion...\", "
                        "\"final_explanation\": \"Final explanation: Based on the analysis, the correct answer is Ethane.\"}\n\n"
                        "Make sure your output is valid JSON."
                    )
                }
            ]
        }
    ]

    # 'processor' and 'model' are assumed to be defined elsewhere in your code
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt"
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    del image
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    output_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]

    # Print only the AI Reasoning Output
    print("\n🔍 **AI Reasoning Output:**\n")
    print(output_text)

    del inputs, output_ids, generated_ids
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return output_text

#####################################
# 5) Process each image & write JSONL output
#####################################
# Open the output file in append mode to save results one-by-one
with open(output_filename, "a", encoding="utf-8") as f_out:
    for filename in sorted(os.listdir(image_folder)):
        if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        image_path = os.path.join(image_folder, filename)

        # Skip if this image has already been processed.
        if image_path in processed_images:
            print(f"Skipping already processed image: {image_path}")
            continue

        # Get the base filename (e.g. "resized_image_0.png")
        base_name = os.path.basename(filename)
        # Remove the "resized_" prefix if present so it matches the original image name
        if base_name.startswith("resized_"):
            base_name = base_name.replace("resized_", "", 1)

        # Reconstruct the original path as stored in the JSON mapping (if needed)
        original_path_in_json = os.path.join("/content/downloaded_images", base_name)
        # # Optionally, you could still get an answer from answers_dict:
        # _ = answers_dict.get(original_path_in_json, "N/A")

        # Generate reasoning (only AI Reasoning Output is printed from within the function)
        # The second argument '_' was previously used but not defined. Modify as needed if you have a specific value.
        ai_output_text = generate_reasoning(image_path, None)
        parsed_data = parse_model_output(ai_output_text)

        # Use the correct answer from the model's output.
        model_correct_answer = parsed_data.get("correct_answer", "N/A")
        if model_correct_answer not in ["A", "B", "C", "D"]:
            final_answer = "N/A"
        else:
            final_answer = model_correct_answer

        # Create a new dictionary with only the correct_answer and image_path
        result = {
            "image_path": image_path,
            "correct_answer": final_answer,
        }

        # Print the result
        print("\nResult:")
        print(json.dumps(result, ensure_ascii=False))

        # Save the result in the output JSONL file.
        f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
        f_out.flush()  # Ensure the result is saved immediately

        # Add the image path to the processed set so it's not processed again in subsequent runs.
        processed_images.add(image_path)

        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

print(f"\n✅ Done processing all images. Results saved in '{output_filename}'.")


Skipping already processed image: /content/downloaded_images/image_0.png
Skipping already processed image: /content/downloaded_images/image_1.png
Skipping already processed image: /content/downloaded_images/image_10.png
Skipping already processed image: /content/downloaded_images/image_100.png
Skipping already processed image: /content/downloaded_images/image_101.png
Skipping already processed image: /content/downloaded_images/image_102.png
Skipping already processed image: /content/downloaded_images/image_103.png
Skipping already processed image: /content/downloaded_images/image_104.png
Skipping already processed image: /content/downloaded_images/image_105.png
Skipping already processed image: /content/downloaded_images/image_106.png
Skipping already processed image: /content/downloaded_images/image_107.png
Skipping already processed image: /content/downloaded_images/image_108.png
Skipping already processed image: /content/downloaded_images/image_109.png
Skipping already processed ima

In [17]:
# prompt: download output_raw.jsonl

from google.colab import files
files.download('output_raw.jsonl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>